In [16]:
import json
import string
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/karen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/karen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/karen/nltk_data...


True

In [4]:
data_file = open('intents.json').read()
data = json.loads(data_file)

In [5]:
data

{'intents': [{'tag': 'hello',
   'patterns': ['Hello', 'Hi there', 'Good morning', "What's up"],
   'responses': ['Hey!', 'Hello', 'Hi!', 'Good morning!'],
   'context': ''},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'job',
   'patterns': ['What is your job', 'What is your work'],
   'responses': ['My job is to make you feel like everything is okay.',
    'I work to serve you as well as possible'],
   'context': ''},
  {'tag': 'age',
   'patterns': ['What is your age', 'How old are you', 'When were you born'],
   'responses': ['I was born in 2021'],
   'context': ''},
  {'tag': 'feeling',
   'patterns': ['How are you today', 'How are you'],
   'responses': ['I am feeling good, you?',
    'Very good and you?',
    "Actually, I'm okay and you?"],
   'context': ''},
  {'tag': 'good',
   'patterns': ['I am good too',
    'I feel fine',
    'Good !',
  

In [11]:
words = []
classes = []
data_X = []
data_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        tokens = nltk.word_tokenize(pattern)
        words.append(tokens)
        data_X.append(pattern)
        data_y.append(intent['tag'])
        
    if intent['tag'] not in classes:
        classes.append(intent['tag'])
        
lemmatizer = WordNetLemmatizer()

# words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
lemmatized_words = []
for word_list in words:
    for word in word_list:
        if word not in string.punctuation:
            lemmatized_words.append(word)

words = sorted(set(lemmatized_words))

classes = sorted(set(classes))

In [17]:
training = []
out_empty = np.zeros(len(classes))

for idx, doc in enumerate(data_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
        
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    
    training.append([bow, output_row])
    
random.shuffle(training)
training = np.array(training, dtype=object)

train_X = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [23]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
adam = tf.keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy']
)

print(model.summary())
model.fit(x=train_X, y=train_y, epochs=150, verbose=1)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 128)               8448      
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 16)                1040      
                                                                 
Total params: 17744 (69.31 KB)
Trainable params: 17744 (69.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/150
2

2/2 [==============================] - 0s 5ms/step - loss: 0.2426 - accuracy: 0.9024
Epoch 74/150
2/2 [==============================] - 0s 7ms/step - loss: 0.2492 - accuracy: 0.9268
Epoch 75/150
2/2 [==============================] - 0s 10ms/step - loss: 0.2065 - accuracy: 0.8780
Epoch 76/150
2/2 [==============================] - 0s 8ms/step - loss: 0.3647 - accuracy: 0.8537
Epoch 77/150
2/2 [==============================] - 0s 10ms/step - loss: 0.1902 - accuracy: 0.9268
Epoch 78/150
2/2 [==============================] - 0s 12ms/step - loss: 0.1730 - accuracy: 0.9268
Epoch 79/150
2/2 [==============================] - 0s 9ms/step - loss: 0.2426 - accuracy: 0.8293
Epoch 80/150
2/2 [==============================] - 0s 10ms/step - loss: 0.2283 - accuracy: 0.9024
Epoch 81/150
2/2 [==============================] - 0s 10ms/step - loss: 0.1638 - accuracy: 0.9268
Epoch 82/150
2/2 [==============================] - 0s 8ms/step - loss: 0.3316 - accuracy: 0.8780
Epoch 83/150
2/2 [==========

In [24]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = np.zeros(len(vocab))
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word == w:
                bow[idx] = 1
    return np.array(bow)

def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocabs)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list, intents_json):
    if len(intents_list) == 0:
        result = "Sorry! I don't understand."
    else:
        tag = intents_list[0]
        list_of_intents = intents_json['intents']
        for i in list_of_intents:
            if i['tag'] == tag:
                result = random.choice(i['responses'])
                break
    return result
    